# exp051_trial

In [1]:
MODE = 'local_train'
#MODE = 'kaggle_inference'

In [2]:
exp_name = 'exp051'
memo = '1st改善 s-bert'

In [3]:
import os
import sys
import gc

if MODE == 'local_train':
    sys.path.append('/home/kaggler/.local/lib/python3.8/site-packages')
    from dotenv import load_dotenv
    load_dotenv
    sys.path.append(os.getenv('UTILS_PATH'))
    import line_notify
    import slack_notify
    
if MODE == "kaggle_inference":
    from cuml import ForestInference
    import treelite
    
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
import matplotlib.pyplot as plt; plt.style.use("ggplot")
import seaborn as sns
from sklearn.metrics.pairwise import haversine_distances
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.decomposition import TruncatedSVD
import lightgbm as lgb
import itertools
from scipy.spatial.distance import canberra
from cuml.neighbors import KNeighborsRegressor
import functools
import multiprocessing
import Levenshtein
import difflib
import pickle
from tqdm import tqdm
%load_ext Cython

from transformers import DistilBertModel, DistilBertTokenizer
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torch.nn as nn
import torch

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

from unidecode import unidecode
import pykakasi

from sentence_transformers import SentenceTransformer

In [4]:
# directry_setting
if MODE == 'local_train':
    INPUT_DIR = os.getenv('INPUT_DIR')
    OUTPUT_DIR = os.getenv('OUTPUT_DIR')
    MODEL_DIR = os.getenv('OUTPUT_DIR')
    BERT_MODEL = "distilbert-base-multilingual-cased"
    os.makedirs(OUTPUT_DIR + exp_name, exist_ok=True)

elif MODE == 'kaggle_inference':
    INPUT_DIR = '/kaggle/input/foursquare-location-matching/'
    OUTPUT_DIR = './'
    MODEL_DIR = f'../input/fs{exp_name}/'
    BERT_MODEL = "../input/distilbertbaseuncased"

In [5]:
# CONFIG
SEED = 42
N_NEIGHBORS = 10
N_SPLITS = 5
PROB_TH = 0.5
MAX_LEN = 32
BS = 512
NW = 2
SVD_N_COMP = 50

In [6]:
class Cat2VecModel(nn.Module):
    def __init__(self):
        super(Cat2VecModel, self).__init__()
        self.distill_bert = DistilBertModel.from_pretrained(BERT_MODEL)
        
    def forward(self, ids, mask):
        x = self.distill_bert(ids, mask)[0]
        x = F.normalize((x[:, 1:, :]*mask[:, 1:, None]).mean(axis=1))
        return x

class InferenceDataset(Dataset):
    
    def __init__(self, df, max_len, col):
        super().__init__()
        self.df = df.reset_index(drop=True)
        self.max_len = max_len
        self.tokenizer = DistilBertTokenizer.from_pretrained(BERT_MODEL, do_lower_case=True)
        self.col = col

    def __getitem__(self, index):
        row = self.df.iloc[index]
        
        inputs = self.tokenizer.encode_plus(
            row[self.col],
            None,
            add_special_tokens=True,
            max_length=MAX_LEN,
            padding="max_length",
            return_token_type_ids=True,
            truncation=True
        )
        ids = torch.LongTensor(inputs['input_ids'])
        mask = torch.LongTensor(inputs['attention_mask'])

        return ids, mask

    def __len__(self):
        return self.df.shape[0]

def inference(ds):
    cat2vec_model = Cat2VecModel()
    cat2vec_model = cat2vec_model.cuda()
    
    loader = DataLoader(ds, batch_size=BS, shuffle=False, num_workers=NW,
                        pin_memory=False, drop_last=False)
    
    vs = []
    with torch.no_grad():
        for idx, (ids, masks) in enumerate(loader):
            v = cat2vec_model(ids.cuda(), masks.cuda()).detach().cpu().numpy()
            vs.append(v)
    return np.concatenate(vs)

In [7]:
def make_bert_vec(df, col):
    cat_df = df[[col]].drop_duplicates()
    cat_df[col] = cat_df[col].fillna("null")

    cat_ds = InferenceDataset(cat_df, max_len=MAX_LEN, col=col)
    V = inference(cat_ds)
    #svd = TruncatedSVD(n_components=SVD_N_COMP, random_state=SEED)
    #V = svd.fit_transform(V)
    V = V.astype("float16")
    bert_vec = {k:v for k,v in zip(cat_df[col].values, V)}
    return bert_vec

In [8]:
def add_vec_sim_features(df, vec, vec_name, col):
    sim_list = []
    for str1, str2 in tqdm(df[[f"{col}", f"match_{col}"]].values.astype(str)):
        if str1!="nan" and str2!="nan":
            sim = dot(vec[str1], vec[str2]) / (norm(vec[str1])*norm(vec[str2]))
        else:
            sim = -1
        sim_list.append(sim)
    df[f"{vec_name}_sim_{col}"] = sim_list
    return df

In [9]:
def preprocess(df):
    columns = ['id', 'name', 'address', 'city', 'state',
        'zip', 'country', 'url', 'phone', 'categories']
    for c in columns:
        if c != "id":
            df[c] = df[c].astype(str).str.lower()

    df[["latitude", "longitude"]] = np.deg2rad(df[["latitude", "longitude"]])
    
    return df

In [10]:
from cuml.feature_extraction.text import TfidfVectorizer as TfidfVectorizer_gpu
import cudf, cuml, cupy
from cuml.neighbors import NearestNeighbors as NearestNeighbors_gpu

In [11]:
def extract_candidate_dist(df):
    dfs = []
    for country, country_df in tqdm(df.groupby("country")):
        country_df = country_df.reset_index(drop=True)
        
        knn = KNeighborsRegressor(n_neighbors=min(len(country_df), N_NEIGHBORS), 
                                    metric='haversine', algorithm="brute")
        knn.fit(country_df[['latitude','longitude']], country_df.index.values)
        nears = knn.kneighbors(country_df[['latitude','longitude']], return_distance=False)
        
        k = min(len(country_df), N_NEIGHBORS)
        country_df['match_id'] = country_df['id'].values[nears[:, :k]].tolist()
        country_df = country_df.explode(['match_id'])
        country_df = country_df.loc[country_df['id'] != country_df['match_id']].copy()
        dfs.append(country_df)
    df = pd.concat(dfs).reset_index(drop=True)
    return df

In [12]:
def extract_candidate_tfidf_sim(df, col):
    dfs = []
    for country, country_df in tqdm(df.groupby("country")):
        country_df = country_df[country_df[col]!="nan"].copy()
        if len(country_df) < 2:
            continue

        country_df = country_df.reset_index(drop=True)
        
        model = TfidfVectorizer(ngram_range=(3,3), analyzer="char_wb", use_idf=True)
        text_embeddings = model.fit_transform(country_df[col].tolist())

        model = NearestNeighbors_gpu(n_neighbors=min(len(country_df), N_NEIGHBORS), algorithm="brute")
        model.fit(text_embeddings)
        nears = model.kneighbors(text_embeddings, return_distance=False)
        
        k = min(len(country_df), N_NEIGHBORS)
        country_df['match_id'] = country_df['id'].values[nears[:, :k]].tolist()
        country_df = country_df.explode(['match_id'])
        country_df = country_df.loc[country_df['id'] != country_df['match_id']].copy()
        dfs.append(country_df)
    df = pd.concat(dfs).reset_index(drop=True)
    return df

In [13]:
def make_sbert_vec(df, col, model_name):
    cat_df = df[[col]].drop_duplicates()
    sbert_model = SentenceTransformer(model_name)
    V = sbert_model.encode(cat_df[col].tolist())
    V = V.astype("float16")
    sbert_vec = {k:v for k,v in zip(cat_df[col].values, V)}
    return sbert_vec

In [14]:
def extract_candidate_sbert_sim(df, col, model_name):
    dfs = []
    vecs = make_sbert_vec(df,col, model_name)
    for country, country_df in tqdm(df.groupby("country")):
        country_df = country_df[country_df[col]!="nan"].copy()
        if len(country_df) < 2:
            continue

        country_df = country_df.reset_index(drop=True)
        text_embeddings = np.vstack([vecs[str_] for str_ in country_df[col].values])
        
        model = NearestNeighbors_gpu(n_neighbors=min(len(country_df), N_NEIGHBORS), algorithm="brute")
        model.fit(text_embeddings)
        nears = model.kneighbors(text_embeddings, return_distance=False)
        
        k = min(len(country_df), N_NEIGHBORS)
        country_df['match_id'] = country_df['id'].values[nears[:, :k]].tolist()
        country_df = country_df.explode(['match_id'])
        country_df = country_df.loc[country_df['id'] != country_df['match_id']].copy()
        dfs.append(country_df)
    df = pd.concat(dfs).reset_index(drop=True)
    return df

In [15]:
def add_orgin_data(df, org_df):
    df = df.merge(org_df.add_prefix('match_'), on='match_id')
    df = df.reset_index(drop=True)
    return df

In [16]:
# https://www.kaggle.com/code/columbia2131/foursquare-iou-metrics
def get_id2poi(input_df: pd.DataFrame) -> dict:
    return dict(zip(input_df['id'], input_df['point_of_interest']))

def get_poi2ids(input_df: pd.DataFrame) -> dict:
    return input_df.groupby('point_of_interest')['id'].apply(set).to_dict()

def get_score(input_df: pd.DataFrame, org_data):
    scores = []
    id2poi = get_id2poi(org_data)
    poi2ids = get_poi2ids(org_data)
    for id_str, matches in zip(input_df['id'].to_numpy(), input_df['matches'].to_numpy()):
        targets = poi2ids[id2poi[id_str]]
        preds = set(matches.split())
        score = len((targets & preds)) / len((targets | preds))
        scores.append(score)
    scores = np.array(scores)
    return scores.mean()

def calc_max_score(tr_data, org_data):
    train_candidate = pd.DataFrame()
    train_candidate['id'] = org_data['id'].unique()
    train_candidate['matches'] = org_data['id'].unique()
    idx = tr_data['point_of_interest']==tr_data['match_point_of_interest']
    train_match = tr_data.loc[idx].groupby('id')['match_id'].apply(list).map(" ".join).reset_index()
    train_match.columns = ['id','candidates']
    train_candidate = train_candidate.merge(train_match, on = 'id', how = 'left')
    idx = ~train_candidate['candidates'].isna()
    train_candidate.loc[idx, "matches"] += " " + train_candidate.loc[idx, "candidates"]
    score = get_score(train_candidate, org_data)
    print('1st_stage_max_score : ' + '{:.5f}'.format(score))
    return score

s-bert

In [17]:
model = SentenceTransformer('LaBSE')
sentences = ["ローソン", "lawson", "セブンイレブン", "7-Eleven"]
embeddings = model.encode(sentences)
print("s-bert(LaBSE)")
print(sentences[0], sentences[1], dot(embeddings[0], embeddings[1]))
print(sentences[0], sentences[2], dot(embeddings[0], embeddings[2]))
print(sentences[0], sentences[3], dot(embeddings[0], embeddings[3]))
print(sentences[1], sentences[2], dot(embeddings[1], embeddings[2]))
print(sentences[2], sentences[3], dot(embeddings[2], embeddings[3]))

s-bert(LaBSE)
ローソン lawson 0.79463166
ローソン セブンイレブン 0.22475721
ローソン 7-Eleven 0.28290558
lawson セブンイレブン 0.19144806
セブンイレブン 7-Eleven 0.66612047


In [19]:
model = SentenceTransformer('stsb-xlm-r-multilingual')
sentences = ["ローソン", "lawson", "セブンイレブン", "7-Eleven"]
embeddings = model.encode(sentences)
print("s-bert(stsb-xlm-r-multilingua)")
print(sentences[0], sentences[1], dot(embeddings[0], embeddings[1])/(norm(embeddings[0])*norm(embeddings[1])))
print(sentences[0], sentences[2], dot(embeddings[0], embeddings[2])/(norm(embeddings[0])*norm(embeddings[2])))
print(sentences[0], sentences[3], dot(embeddings[0], embeddings[3])/(norm(embeddings[0])*norm(embeddings[3])))
print(sentences[1], sentences[2], dot(embeddings[1], embeddings[2])/(norm(embeddings[1])*norm(embeddings[2])))
print(sentences[2], sentences[3], dot(embeddings[2], embeddings[3])/(norm(embeddings[2])*norm(embeddings[3])))

s-bert(stsb-xlm-r-multilingua)
ローソン lawson 0.66890115
ローソン セブンイレブン 0.81569093
ローソン 7-Eleven 0.43940908
lawson セブンイレブン 0.66826326
セブンイレブン 7-Eleven 0.5715176


In [20]:
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
sentences = ["ローソン", "lawson", "セブンイレブン", "7-Eleven"]
embeddings = model.encode(sentences)
print("s-bert(paraphrase-multilingual-mpnet-base-v2)")
print(sentences[0], sentences[1], dot(embeddings[0], embeddings[1])/(norm(embeddings[0])*norm(embeddings[1])))
print(sentences[0], sentences[2], dot(embeddings[0], embeddings[2])/(norm(embeddings[0])*norm(embeddings[2])))
print(sentences[0], sentences[3], dot(embeddings[0], embeddings[3])/(norm(embeddings[0])*norm(embeddings[3])))
print(sentences[1], sentences[2], dot(embeddings[1], embeddings[2])/(norm(embeddings[1])*norm(embeddings[2])))
print(sentences[2], sentences[3], dot(embeddings[2], embeddings[3])/(norm(embeddings[2])*norm(embeddings[3])))

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/723 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/402 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

s-bert(paraphrase-multilingual-mpnet-base-v2)
ローソン lawson 0.5309868
ローソン セブンイレブン 0.6825693
ローソン 7-Eleven 0.291601
lawson セブンイレブン 0.500501
セブンイレブン 7-Eleven 0.325499


In [18]:
df = pd.DataFrame()
org_df = pd.DataFrame()
org_df["name"] = ["ローソン", "lawson", "セブンイレブン", "7-Eleven"]
df["name"] = ["ローソン", "ローソン", "ローソン", "lawson", "セブンイレブン"]
df["match_name"] = ["lawson", "セブンイレブン", "7-Eleven", "セブンイレブン", "7-Eleven"]
vecs = make_bert_vec(org_df, "name")
df = add_vec_sim_features(df, vecs, "bert", "name")
print("bert")
df

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 5/5 [00:00<00:00, 10754.63it/s]

bert


,name,match_name,bert_sim_name
0,ローソン,lawson,0.515625
1,ローソン,セブンイレブン,0.722168
2,ローソン,7-Eleven,0.426758
3,lawson,セブンイレブン,0.499268
4,セブンイレブン,7-Eleven,0.467041


実験

In [19]:
train_origin = pd.read_csv(INPUT_DIR + "train.csv")
train_origin = preprocess(train_origin)

# trainデータの分割
kf = GroupKFold(n_splits=2)
for i, (trn_idx, val_idx) in enumerate(kf.split(train_origin, train_origin['point_of_interest'], train_origin['point_of_interest'])):
    train_origin = train_origin.loc[val_idx].copy()
    break
train_origin = train_origin.reset_index(drop=True)

In [20]:
conds = []
scores = []

In [21]:
cond = "tfidf"
train = extract_candidate_tfidf_sim(train_origin, "name")
train = train.merge(train_origin[["id", "point_of_interest"]].add_prefix("match_"), on="match_id", how="left")
score = calc_max_score(train, train_origin)
conds.append(cond)
scores.append(score)

100%|██████████| 210/210 [00:36<00:00,  5.74it/s]


1st_stage_max_score : 0.88422


In [22]:
cond = "sbert(LaBSE)"
train = extract_candidate_sbert_sim(train_origin, "name", "LaBSE")
train = train.merge(train_origin[["id", "point_of_interest"]].add_prefix("match_"), on="match_id", how="left")
score = calc_max_score(train, train_origin)
conds.append(cond)
scores.append(score)

 92%|█████████▏| 193/210 [00:48<00:04,  4.01it/s]


RuntimeError: Error in virtual void faiss::gpu::StandardGpuResourcesImpl::initializeForDevice(int) at _deps/faiss-src/faiss/gpu/StandardGpuResources.cpp:277: Error: 'err == cudaSuccess' failed: failed to cudaHostAlloc 268435456 bytes for CPU <-> GPU async copy buffer (error 2 out of memory)

In [ ]:
result = pd.DataFrame()
result["条件"] = conds
result["1st_max_score"] = scores
result